# GAMA-12 master catalogue
## Preparation of UKIRT Infrared Deep Sky Survey / Large Area Survey (UKIDSS/LAS)

Information about UKIDSS can be found at http://www.ukidss.org/surveys/surveys.html

The catalogue comes from `dmu0_UKIDSS-LAS`.

In the catalogue, we keep:

- The identifier (it's unique in the catalogue);
- The position;
- The stellarity;
- The magnitude for each band in aperture 3 (2 arcsec).
- The hall magnitude is described as the total magnitude.

J band magnitudes are available in two eopchs. We take the first arbitrarily.

The magnitudes are “*Vega like*”.  The AB offsets are given by Hewett *et al.* (2016):

| Band | AB offset |
|------|-----------|
| Y    | 0.634     |
| J    | 0.938     |
| H    | 1.379     |
| K    | 1.900     |

Each source is associated with an epoch. These range between 2005 and 2007. We take 2006 for the epoch.

In [ ]:
from herschelhelp_internal import git_version
print("This notebook was run with herschelhelp_internal version: \n{}".format(git_version()))

In [ ]:
%matplotlib inline
#%config InlineBackend.figure_format = 'svg'

import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))

from collections import OrderedDict
import os

from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.table import Column, Table
import numpy as np

from herschelhelp_internal.flagging import  gaia_flag_column
from herschelhelp_internal.masterlist import nb_astcor_diag_plot, remove_duplicates
from herschelhelp_internal.utils import astrometric_correction, mag_to_flux

In [ ]:
OUT_DIR =  os.environ.get('TMP_DIR', "./data_tmp")
try:
    os.makedirs(OUT_DIR)
except FileExistsError:
    pass

RA_COL = "las_ra"
DEC_COL = "las_dec"

## I - Column selection

In [ ]:
#Is the following standard (different names for radec vs mag)?
imported_columns = OrderedDict({
        'SOURCEID': 'las_id',
        'RA': 'las_ra',
        'Dec': 'las_dec',
        'YHALLMAG': 'm_ukidss_y',
        'YHALLMAGERR': 'merr_ukidss_y',
        'YAPERMAG3': 'm_ap_ukidss_y',
        'YAPERMAG3ERR': 'merr_ap_ukidss_y',
        'J_1HALLMAG': 'm_ukidss_j',
        'J_1HALLMAGERR': 'merr_ukidss_j',
        'J_1APERMAG3': 'm_ap_ukidss_j',
        'J_1APERMAG3ERR': 'merr_ap_ukidss_j',
        'HAPERMAG3': 'm_ap_ukidss_h',
        'HAPERMAG3ERR': 'merr_ap_ukidss_h',
        'HHALLMAG': 'm_ukidss_h',
        'HHALLMAGERR': 'merr_ukidss_h',
        'KAPERMAG3': 'm_ap_ukidss_k',
        'KAPERMAG3ERR': 'merr_ap_ukidss_k',
        'KHALLMAG': 'm_ukidss_k',
        'KHALLMAGERR': 'merr_ukidss_k',
        'PSTAR': 'las_stellarity'
    })

catalogue = Table.read(
    "../../dmu0/dmu0_UKIDSS-LAS/data/UKIDSS-LAS_GAMA-12.fits")[list(imported_columns)]
for column in imported_columns:
    catalogue[column].name = imported_columns[column]

#Epochs between 2005 and 2007. Rough average:
epoch = 2006

# Clean table metadata
catalogue.meta = None

In [ ]:
# Adding flux and band-flag columns
for col in catalogue.colnames:
    if col.startswith('m_'):
        
        errcol = "merr{}".format(col[1:])
        
        # LAS uses a huge negative number for missing values
        catalogue[col][catalogue[col] < -100] = np.nan
        catalogue[errcol][catalogue[errcol] < -100] = np.nan        

        # Vega to AB correction
        if col.endswith('y'):
            catalogue[col] += 0.634
        elif col.endswith('j'):
            catalogue[col] += 0.938
        elif col.endswith('h'):
            catalogue[col] += 1.379
        elif col.endswith('k'):
            catalogue[col] += 1.900
        else:
            print("{} column has wrong band...".format(col))
        
        flux, error = mag_to_flux(np.array(catalogue[col]), np.array(catalogue[errcol]))
        
        # Fluxes are added in µJy
        catalogue.add_column(Column(flux * 1.e6, name="f{}".format(col[1:])))
        catalogue.add_column(Column(error * 1.e6, name="f{}".format(errcol[1:])))
        
        # Band-flag column
        if "ap" not in col:
            catalogue.add_column(Column(np.zeros(len(catalogue), dtype=bool), name="flag{}".format(col[1:])))
        
# TODO: Set to True the flag columns for fluxes that should not be used for SED fitting.

In [ ]:
catalogue[:10].show_in_notebook()

## II - Removal of duplicated sources

We remove duplicated objects from the input catalogues.

In [ ]:
SORT_COLS = ['merr_ap_ukidss_j', 'merr_ap_ukidss_k']
FLAG_NAME = 'las_flag_cleaned'

nb_orig_sources = len(catalogue)

catalogue = remove_duplicates(catalogue, RA_COL, DEC_COL, sort_col=SORT_COLS, flag_name=FLAG_NAME)

nb_sources = len(catalogue)

print("The initial catalogue had {} sources.".format(nb_orig_sources))
print("The cleaned catalogue has {} sources ({} removed).".format(nb_sources, nb_orig_sources - nb_sources))
print("The cleaned catalogue has {} sources flagged as having been cleaned".format(np.sum(catalogue[FLAG_NAME])))

## III - Astrometry correction

We match the astrometry to the Gaia one. We limit the Gaia catalogue to sources with a g band flux between the 30th and the 70th percentile. Some quick tests show that this give the lower dispersion in the results.

In [ ]:
gaia = Table.read("../../dmu0/dmu0_GAIA/data/GAIA_GAMA-12.fits")
gaia_coords = SkyCoord(gaia['ra'], gaia['dec'])

In [ ]:
nb_astcor_diag_plot(catalogue[RA_COL], catalogue[DEC_COL], 
                    gaia_coords.ra, gaia_coords.dec, near_ra0=True)

In [ ]:
delta_ra, delta_dec =  astrometric_correction(
    SkyCoord(catalogue[RA_COL], catalogue[DEC_COL]),
    gaia_coords, near_ra0=True
)

print("RA correction: {}".format(delta_ra))
print("Dec correction: {}".format(delta_dec))

In [ ]:
catalogue[RA_COL] +=  delta_ra.to(u.deg)
catalogue[DEC_COL] += delta_dec.to(u.deg)

In [ ]:
nb_astcor_diag_plot(catalogue[RA_COL], catalogue[DEC_COL], 
                    gaia_coords.ra, gaia_coords.dec, near_ra0=True)

## IV - Flagging Gaia objects

In [ ]:
catalogue.add_column(
    gaia_flag_column(SkyCoord(catalogue[RA_COL], catalogue[DEC_COL]), epoch, gaia)
)

In [ ]:
GAIA_FLAG_NAME = "las_flag_gaia"

catalogue['flag_gaia'].name = GAIA_FLAG_NAME
print("{} sources flagged.".format(np.sum(catalogue[GAIA_FLAG_NAME] > 0)))

## V - Flagging objects near bright stars

# VI - Saving to disk

In [ ]:
catalogue.write("{}/UKIDSS-LAS.fits".format(OUT_DIR), overwrite=True)